In [1]:
import polars as pl
import re

In [26]:
retailers_with_id = [
        "coup_de_pates",
        "ds_restauration",
        "even",
        "metro",
        "pomona",
        "pro_a_pro",
        "sysco",
        "transgourmet",
    ]
retailers_without_id = [
        "ducreux",
        "relais_dor",
    ]

retailers = retailers_with_id + retailers_without_id

# Extract products sample

In [ ]:
datasets = []

for retailer_with_id in retailers_with_id:
    dataset = pl.read_parquet(f'data/silver/with_id/{retailer_with_id}.parquet')
    dataset = dataset.unique(subset=[c for c in dataset.columns if c.startswith('level') or c == 'volume_unit']).with_columns(pl.lit(retailer_with_id).alias('retailer'))
    datasets.append(dataset)

dataset_concat = pl.concat(datasets)
dataset_concat = dataset_concat.sort(['retailer']+ [c for c in dataset_concat.columns if c.startswith('level')] + ['volume_unit'])

dataset_concat.write_csv('data/log/model_4/products_subset_withId.csv', separator=';')

In [ ]:
datasets = []

for retailer_without_id in retailers_without_id:
    dataset = pl.read_parquet(f'data/silver/without_id/{retailer_without_id}.parquet')
    dataset = dataset.unique(subset=[c for c in dataset.columns if c.startswith('level') or c == 'volume_unit']).with_columns(pl.lit(retailer_without_id).alias('retailer'))
    datasets.append(dataset)

dataset_concat = pl.concat(datasets)
dataset_concat = dataset_concat.sort(['retailer']+ [c for c in dataset_concat.columns if c.startswith('level')] + ['volume_unit'])

dataset_concat.write_csv('data/log/model_4/products_subset_withoutId.csv', separator=';')

# Extract volume unit

In [ ]:
datasets = []

for retailer in retailers:
    if retailer in retailers_with_id:
        dataset = pl.read_parquet(f'data/silver/with_id/{retailer}.parquet')
    else :
        dataset = pl.read_parquet(f'data/silver/without_id/{retailer}.parquet')

    dataset = dataset.select('volume_unit').unique().with_columns(pl.lit(retailer).alias('retailer'))
    datasets.append(dataset)

dataset_concat = pl.concat(datasets).select('retailer', 'volume_unit').sort('retailer')
dataset_concat.write_csv('data/log/model_4/volume_unit_subset.csv', separator=';')

In [17]:
import xlsxwriter

from src.utils.profiling import incremental_sublists

retailers_with_id = [
        "coup_de_pates",
        "ds_restauration",
        "even",
        "metro",
        "pomona",
        "pro_a_pro",
        "sysco",
        "transgourmet",
    ]
retailers_without_id = [
        "ducreux",
        "relais_dor",
    ]

retailers = retailers_with_id + retailers_without_id

wb = xlsxwriter.Workbook(
            f"data/log/model_4/volume_unit_subset_detailed.xlsx"
        )


agg_cols_lst = incremental_sublists([f'level_{i}_standard' for i in range(1, 5)])
for i, agg_cols in enumerate(agg_cols_lst, start=1):

    datasets = []
    for retailer in retailers:
        if retailer in retailers_with_id:
            dataset = pl.read_parquet(f'data/silver/with_id/{retailer}.parquet').rename(lambda column_name: column_name.replace(f"_{retailer}", ""))
            product_col = 'product_id'
        else :
            dataset = pl.read_parquet(f'data/silver/without_id/{retailer}.parquet').rename(lambda column_name: column_name.replace(f"_{retailer}", ""))
            product_col = 'product_code'

        dataset = dataset.group_by(['volume_unit'] + agg_cols).agg(pl.col(product_col).n_unique().alias('nb_products')).unique().with_columns(pl.lit(retailer).alias('retailer')).sort(agg_cols + ['retailer', 'volume_unit'])
        datasets.append(dataset)

    dataset_concat = pl.concat(datasets)
    dataset_concat.write_excel(workbook=wb, worksheet=f"level_{i}")

wb.close()

# Volume from product name

In [5]:
texts = [
    "LASAGNE AUX OEUFS BOITE 500 G BARILLA",
    "LASAGNE AUX OEUFS BOITE 500G BARILLA",
    "LASAGNE AUX OEUFS BOITE 500 G",
    "LASAGNE AUX OEUFS BOITE 500G",
    "LASAGNE AUX OEUFS BOITE 500 G x120",
    "LASAGNE AUX OEUFS BOITE 500G X 120",
    "LASAGNE AUX OEUFS BOITE 500.0 L",
    "LASAGNE AUX OEUFS BOITE 500,11 CL 500 ml",
    "LASAGNE AUX OEUFS BOITE 500,11 CL 500 ml x100",
    "LASAGNE AUX OEUFS BOITE 500,11 CL 500 ml x100 BARILLA",
    "LASAGNE AUX OEUFS BOITE 500,11 CL 500 ml x100BARILLA"
]

pattern = r"\b(\d+(?:\.|,)*\d*)\s*(g|kg|ml|cl|l|gr)\b\s*(x\s*\d+)?\b"

for text in texts:
    match = re.findall(pattern, text, re.IGNORECASE)  # Case-insensitive matching
#     match = re.search(pattern, text, re.IGNORECASE)  # Case-insensitive matching
    if match:
        print(match)
        # print(match.group().upper())  # Extracted value


"""What This Regex Does
\b → Ensures word boundary (matches only whole words).
(\d+(?:\.|,)*\d*) → Captures a number with optional decimal/comma separators (e.g., "500", "1.5", "2,5").
\s* → Allows optional spaces between number and unit.
(g|kg|ml|cl|l|gr) → Captures common measurement units.
\b → Ensures the unit is a whole word.
\s*(x\s*\d+)?\b → Optionally captures a multiplier format (e.g., "x 2" in "500 G x 2")."""

[('500', 'G', '')]
[('500', 'G', '')]
[('500', 'G', '')]
[('500', 'G', '')]
[('500', 'G', 'x120')]
[('500', 'G', 'X 120')]
[('500.0', 'L', '')]
[('500,11', 'CL', ''), ('500', 'ml', '')]
[('500,11', 'CL', ''), ('500', 'ml', 'x100')]
[('500,11', 'CL', ''), ('500', 'ml', 'x100')]
[('500,11', 'CL', ''), ('500', 'ml', '')]


'What This Regex Does\n\x08 → Ensures word boundary (matches only whole words).\n(\\d+(?:\\.|,)*\\d*) → Captures a number with optional decimal/comma separators (e.g., "500", "1.5", "2,5").\n\\s* → Allows optional spaces between number and unit.\n(g|kg|ml|cl|l|gr) → Captures common measurement units.\n\x08 → Ensures the unit is a whole word.\n\\s*(x\\s*\\d+)?\x08 → Optionally captures a multiplier format (e.g., "x 2" in "500 G x 2").'

In [6]:
# # pl.Config.set_fmt_str_lengths(100)

# def extract_volume(text:str)->list:
#     pattern = r"\b(\d+(?:\.|,)*\d*)\s*(g|kg|ml|cl|l|gr)\b\s*(x\s*\d+)?\b"
#     matches = re.findall(pattern, text, re.IGNORECASE)
#     return matches if matches else []

# retailer = 'coup_de_pates'
# with pl.Config(tbl_rows=-1):
#     display(
#         pl.read_parquet(f'data/silver/with_id/{retailer}.parquet')
#         .filter(pl.col('volume_unit_even')=="UNT  ")
#         # .filter(pl.col('level_3_standard_even')=='PATE')
#         .with_columns(pl.col(f'product_name_{retailer}').map_elements(lambda x : extract_volume(x)).alias('new_volume_unit'))
#         .select(f'product_name_{retailer}', 'new_volume_unit')
#         # .filter(pl.col('new_volume_unit')==[])
#         .sample(n=40)
#         )
# # .filter(pl.col('level_3_standard_even')=='PATE')

# Volume <=0

In [52]:
data = []
for retailer in retailers:
    if retailer in retailers_with_id:
        dataset = pl.read_parquet(f'data/silver/with_id/{retailer}.parquet')
    else :
        dataset = pl.read_parquet(f'data/silver/without_id/{retailer}.parquet')

    if dataset.select("volume").dtypes[0] == pl.String:
        dataset = dataset.with_columns(
            pl.col("volume").str.replace(",", ".").str.replace(" ", "").cast(pl.Float64)
        )

    nb_rows_v0tp = dataset.filter((pl.col('volume')==0)|(pl.col('volume').is_null())).filter(pl.col('turnover')>0).shape[0]
    nb_rows_v0tn = dataset.filter((pl.col('volume')==0)|(pl.col('volume').is_null())).filter(pl.col('turnover')<0).shape[0]
    nb_rows_v0t0 = dataset.filter((pl.col('volume')==0)|(pl.col('volume').is_null())).filter((pl.col('turnover')==0)|(pl.col('turnover').is_null())).shape[0]
    nb_rows_vntp = dataset.filter((pl.col('volume')<0)).filter(pl.col('turnover')>0).shape[0]
    nb_rows_vntn = dataset.filter((pl.col('volume')<0)).filter(pl.col('turnover')<0).shape[0]
    nb_rows_vptn = dataset.filter((pl.col('volume')>0)).filter(pl.col('turnover')<0).shape[0]
    data.append(
        {'retailer':retailer,
         'v0tp': nb_rows_v0tp,
         'v0tn': nb_rows_v0tn,
         'v0t0': nb_rows_v0t0,
         'vntp': nb_rows_vntp,
         'vntn': nb_rows_vntn,
         'vptn': nb_rows_vptn
        })

    # display(dataset.filter((pl.col('volume')<0)).filter(pl.col('turnover')>0).sample(n=1))
res = pl.from_dicts(data)
display(res)
res.write_excel('volume_turnover_eda.xlsx')

retailer,v0tp,v0tn,v0t0,vntp,vntn,vptn
str,i64,i64,i64,i64,i64,i64
"""coup_de_pates""",38,88,1,4,615,4
"""ds_restauration""",13,19,2069,2,104,6
"""even""",51,46,1113,5,879,10
"""metro""",14111,489,4301,21,3479,71
"""pomona""",468,736,12365,31,4750,105
"""pro_a_pro""",231,715,0,17,4246,66
"""sysco""",219,926,532932,5,2695,57
"""transgourmet""",264,166,5832,10,1943,35
"""ducreux""",40,79,1964,0,557,6


# EDA

- coup_de_pates 
    - [Poids N] : volume vendu
    - Unité Poids : unité de volume
-> que en KG

- ds_restauration
    - Volume Vendu (kg) : volume vendu
    - unité de volume : kg
-> en KG/L

- ducreux
    - Poids net liv: volume vendu 
-> A transformer en KG/L

- even 
    - Poids Saisi Livré: volume vendu en kg/L
    - Qté / Poids Livré: volume en Unité de Facturation (kg, unité, L, ...)
    - Unité de Facturation: volume_unit
-> en KG/L
Produits exprimés en litre en kg chez un autre 

- metro
    - Poids_volume: volume
    - unite_volume : unité de volume
-> A transformer en KG/L + manquants

- pomona
    - Poids livré 
-> en KG/L ? 

- pro_a_pro
    - Volume facturé: volume vendu
-> en KG/L

- relais_dor
    - Poids 
-> en KG/L ? 

- sysco
    - Quantité KG: volume vendu en kg
-> en KG/L ? 

- transgourmet
    - Poids Brut Facturé
-> en KG/L ? 

In [ ]:
even = (
    pl.read_parquet("data/silver/with_id/even.parquet").select("product_id", 'product_name', 'volume', 'volume_unit')
    .rename(lambda col: f'{col}_even')
    .rename({f'product_id_even': "product_id"})
)

metro = (
    pl.read_parquet("data/silver/with_id/metro.parquet").select("product_id", 'product_name', 'volume', 'volume_unit')
    .rename(lambda col: f'{col}_metro')
    .rename({f'product_id_metro': "product_id"})
    .with_columns(pl.col('volume_metro').cast(pl.Utf8).str.replace(",", ".").cast(pl.Float64))
)

res = even.join(metro, on='product_id', how='inner').filter(pl.col('volume_unit_even')!=pl.col('volume_unit_metro')).unique(subset=['product_id'])
res.with_columns(pl.col("product_id").cast(pl.Utf8)).write_excel('volume_unit_inconsistencies.xlsx')